In [3]:
# Install required packages (uncomment and run in a separate cell if not already installed)
# !pip install opencv-python mediapipe pygame numpy

import cv2
import numpy as np
import pygame
import mediapipe as mp

class HandPiano:
    def __init__(self):
        # Initialize Mediapipe Hand detection
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(
            static_image_mode=False,
            max_num_hands=1,
            min_detection_confidence=0.7
        )
        self.mp_draw = mp.solutions.drawing_utils

        # Initialize Pygame mixer for audio
        pygame.mixer.init()
        
        # Define note frequencies (you can use these with synthesized sounds)
        self.frequencies = {
            0: 261.63,  # C4
            1: 293.66,  # D4
            2: 329.63,  # E4
            3: 349.23,  # F4
            4: 392.00,  # G4
            5: 440.00,  # A4
            6: 493.88,  # B4
            7: 523.25   # C5
        }
        
        # Create synthesized notes
        self.notes = self.create_synthesized_notes()
        self.last_note = None

    def create_synthesized_notes(self):
        notes = {}
        sample_rate = 44100
        duration = 0.5  # seconds
    
        for note_idx, freq in self.frequencies.items():
            t = np.linspace(0, duration, int(sample_rate * duration))
            # Create a simple sine wave
            note = np.sin(2 * np.pi * freq * t)
            # Apply envelope to avoid clicks
            envelope = np.exp(-t * 3)
            note = note * envelope
            # Normalize and convert to 16-bit integer
            note = (note * 32767).astype(np.int16)
            # Convert to 2D array for stereo sound
            note_stereo = np.stack((note, note), axis=-1)
            # Create sound object
            sound = pygame.sndarray.make_sound(note_stereo)
            notes[note_idx] = sound
        
        return notes


    def get_note_from_position(self, x, width):
        section_width = width // 8
        note_index = int(x // section_width)
        return min(max(note_index, 0), 7)

    def process_frame(self, frame):
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.hands.process(rgb_frame)
        
        height, width, _ = frame.shape
        
        # Draw piano keys
        for i in range(8):
            x1 = i * (width // 8)
            x2 = (i + 1) * (width // 8)
            cv2.rectangle(frame, (x1, height - 100), (x2, height), (255, 255, 255), -1)
            cv2.rectangle(frame, (x1, height - 100), (x2, height), (0, 0, 0), 2)
            # Add note labels
            note_names = ['C4', 'D4', 'E4', 'F4', 'G4', 'A4', 'B4', 'C5']
            cv2.putText(frame, note_names[i], (x1 + 20, height - 20),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                self.mp_draw.draw_landmarks(frame, hand_landmarks, 
                                          self.mp_hands.HAND_CONNECTIONS)
                
                # Get index fingertip position
                index_tip = hand_landmarks.landmark[8]
                x = int(index_tip.x * width)
                y = int(index_tip.y * height)
                
                cv2.circle(frame, (x, y), 10, (0, 255, 0), -1)
                
                if y > height - 100:
                    note_index = self.get_note_from_position(x, width)
                    if self.last_note != note_index:
                        if self.last_note is not None:
                            self.notes[self.last_note].stop()
                        self.notes[note_index].play()
                        self.last_note = note_index
                        
        return frame

    def run(self):
        cap = cv2.VideoCapture(0)
        
        try:
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                    
                frame = cv2.flip(frame, 1)
                frame = self.process_frame(frame)
                
                # Add instructions
                cv2.putText(frame, "Move your index finger over the piano keys to play!", 
                           (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
                
                # Display the frame
                cv2.imshow('Hand Piano', frame)
                
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                    
        finally:
            cap.release()
            cv2.destroyAllWindows()
            pygame.mixer.quit()

# Run the Hand Piano
piano = HandPiano()
piano.run()


In [6]:
import numpy as np
from scipy.io.wavfile import write

# Function to generate a sine wave
def generate_sine_wave(frequency, duration, sample_rate=44100):
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    wave = 0.5 * np.sin(2 * np.pi * frequency * t)  # 0.5 to reduce amplitude
    return (wave * 32767).astype(np.int16)  # Convert to 16-bit PCM

# Define frequencies for each finger sound
frequencies = {
    "thumb": 261.63,  # C4
    "index": 329.63,  # E4
    "middle": 392.00,  # G4
    "ring": 440.00,  # A4
    "pinky": 523.25  # C5
}

# Generate and save .wav files
output_dir = "finger_sounds"
output_files = {}

for finger, freq in frequencies.items():
    wave = generate_sine_wave(freq, duration=1.0)  # 1 second duration
    output_path = f"{output_dir}/{finger}.wav"
    write(output_path, 44100, wave)
    output_files[finger] = output_path

output_files


{'thumb': 'finger_sounds/thumb.wav',
 'index': 'finger_sounds/index.wav',
 'middle': 'finger_sounds/middle.wav',
 'ring': 'finger_sounds/ring.wav',
 'pinky': 'finger_sounds/pinky.wav'}

In [8]:
import os

# Create directory if it doesn't exist
output_dir = "/mnt/data/finger_sounds"
os.makedirs(output_dir, exist_ok=True)

# Regenerate and save .wav files
for finger, freq in frequencies.items():
    wave = generate_sine_wave(freq, duration=1.0)  # 1 second duration
    output_path = f"{output_dir}/{finger}.wav"
    write(output_path, 44100, wave)
    output_files[finger] = output_path

output_files


{'thumb': '/mnt/data/finger_sounds/thumb.wav',
 'index': '/mnt/data/finger_sounds/index.wav',
 'middle': '/mnt/data/finger_sounds/middle.wav',
 'ring': '/mnt/data/finger_sounds/ring.wav',
 'pinky': '/mnt/data/finger_sounds/pinky.wav'}

In [10]:
import cv2
import pygame
import mediapipe as mp
from pygame import mixer

class HandSounds:
    def __init__(self):
        # Initialize Mediapipe Hand detection
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(
            static_image_mode=False,
            max_num_hands=1,
            min_detection_confidence=0.7
        )
        self.mp_draw = mp.solutions.drawing_utils

        # Initialize Pygame mixer for audio
        pygame.mixer.init()
        
        # Load sounds for each finger
        self.finger_sounds = {
            4: pygame.mixer.Sound("thumb.wav"),  # Replace with the path to your sound file
            8: pygame.mixer.Sound("index.wav"),  # Replace with the path to your sound file
            12: pygame.mixer.Sound("middle.wav"),  # Replace with the path to your sound file
            16: pygame.mixer.Sound("ring.wav"),  # Replace with the path to your sound file
            20: pygame.mixer.Sound("pinky.wav"),  # Replace with the path to your sound file
        }
        # Track the state of the last played sounds
        self.last_played = {4: False, 8: False, 12: False, 16: False, 20: False}

    def process_frame(self, frame):
        # Convert the frame to RGB for Mediapipe
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.hands.process(rgb_frame)
        
        height, width, _ = frame.shape
        
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Draw hand landmarks on the frame
                self.mp_draw.draw_landmarks(frame, hand_landmarks, 
                                            self.mp_hands.HAND_CONNECTIONS)
                
                # Iterate over the fingertips
                for landmark_id in self.finger_sounds.keys():
                    fingertip = hand_landmarks.landmark[landmark_id]
                    x = int(fingertip.x * width)
                    y = int(fingertip.y * height)
                    
                    # Visualize the fingertip as a circle
                    cv2.circle(frame, (x, y), 10, (0, 255, 0), -1)
                    
                    # Play sound if fingertip moves into the bottom half of the screen
                    if y > height // 2:
                        if not self.last_played[landmark_id]:
                            self.finger_sounds[landmark_id].play()
                            self.last_played[landmark_id] = True
                    else:
                        self.last_played[landmark_id] = False

        return frame

    def run(self):
        # Start the webcam
        cap = cv2.VideoCapture(0)
        
        try:
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                    
                # Flip the frame horizontally for a mirror effect
                frame = cv2.flip(frame, 1)
                # Process the frame to play sounds based on finger positions
                frame = self.process_frame(frame)
                
                # Add instructions to the frame
                cv2.putText(frame, "Move fingers down to play sounds!", 
                            (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                
                # Display the processed frame
                cv2.imshow('Hand Sounds', frame)
                
                # Quit the application when 'q' is pressed
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                    
        finally:
            # Release resources
            cap.release()
            cv2.destroyAllWindows()
            pygame.mixer.quit()

# Run the program
if __name__ == "__main__":
    hand_sounds = HandSounds()
    hand_sounds.run()


FileNotFoundError: No file 'thumb.wav' found in working directory 'C:\Users\bhuva\Anaconda'.

In [12]:
import cv2
import numpy as np
import pygame
import mediapipe as mp

class HandSounds:
    def __init__(self):
        # Initialize Mediapipe Hand detection
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(
            static_image_mode=False,
            max_num_hands=1,
            min_detection_confidence=0.7
        )
        self.mp_draw = mp.solutions.drawing_utils

        # Initialize Pygame mixer for sound playback
        pygame.mixer.init()

        # Predefined frequencies for fingers
        self.finger_frequencies = {
            "thumb": 261.63,  # C4
            "index": 329.63,  # E4
            "middle": 392.00,  # G4
            "ring": 440.00,  # A4
            "pinky": 523.25  # C5
        }

        # Create sound objects for each finger
        self.sounds = self.generate_sounds()
        self.last_played_finger = None

    def generate_sounds(self):
        """Generate sound objects dynamically for each finger."""
        sounds = {}
        for finger, freq in self.finger_frequencies.items():
            wave = self.generate_sine_wave(freq, duration=0.5)  # 0.5 second duration
            sounds[finger] = pygame.sndarray.make_sound(wave)
        return sounds

    @staticmethod
    def generate_sine_wave(frequency, duration, sample_rate=44100):
        """Generate a sine wave."""
        t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
        wave = 0.5 * np.sin(2 * np.pi * frequency * t)  # 0.5 to reduce amplitude

        # Convert to stereo: create a 2D array with identical left and right channels
        stereo_wave = np.stack((wave, wave), axis=-1)

        return (stereo_wave * 32767).astype(np.int16)  # Convert to 16-bit PCM

    def play_sound(self, finger):
        """Play the sound corresponding to the given finger."""
        if finger in self.sounds and self.last_played_finger != finger:
            self.sounds[finger].play()
            self.last_played_finger = finger

    def detect_finger(self, landmarks):
        """Detect which finger is being pointed."""
        if not landmarks:
            return None

        finger_tips = {
            "thumb": 4,
            "index": 8,
            "middle": 12,
            "ring": 16,
            "pinky": 20
        }

        # Check if fingers are near the bottom of the frame (as a trigger area)
        for finger, tip_id in finger_tips.items():
            if landmarks[tip_id].y > 0.8:  # Bottom 20% of the screen
                return finger
        return None

    def process_frame(self, frame):
        """Process each video frame for hand detection and sound playing."""
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.hands.process(rgb_frame)

        height, width, _ = frame.shape

        # Draw trigger zone
        cv2.rectangle(frame, (0, int(height * 0.8)), (width, height), (200, 200, 200), -1)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                self.mp_draw.draw_landmarks(frame, hand_landmarks, self.mp_hands.HAND_CONNECTIONS)

                # Get finger tip landmarks
                landmarks = hand_landmarks.landmark
                finger = self.detect_finger(landmarks)
                if finger:
                    self.play_sound(finger)

        return frame

    def run(self):
        """Start the webcam and play sounds based on hand gestures."""
        cap = cv2.VideoCapture(0)

        try:
            while True:
                ret, frame = cap.read()
                if not ret:
                    break

                frame = cv2.flip(frame, 1)
                frame = self.process_frame(frame)

                # Add instructions
                cv2.putText(frame, "Move fingers to the bottom area to play sounds", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)

                # Display the frame
                cv2.imshow('Hand Sounds', frame)

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

        finally:
            cap.release()
            cv2.destroyAllWindows()
            pygame.mixer.quit()


# Run the HandSounds application
if __name__ == "__main__":
    hand_sounds = HandSounds()
    hand_sounds.run()


In [13]:
import cv2
import numpy as np
import pygame
import mediapipe as mp

class HandSounds:
    def __init__(self):
        # Initialize Mediapipe Hand detection
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(
            static_image_mode=False,
            max_num_hands=1,
            min_detection_confidence=0.7
        )
        self.mp_draw = mp.solutions.drawing_utils

        # Initialize Pygame mixer for sound playback
        pygame.mixer.init()

        # Predefined frequencies for fingers
        self.finger_frequencies = {
            "thumb": 261.63,  # C4
            "index": 329.63,  # E4
            "middle": 392.00,  # G4
            "ring": 440.00,  # A4
            "pinky": 523.25  # C5
        }

        # Create sound objects for each finger
        self.sounds = self.generate_sounds()

    def generate_sounds(self):
        """Generate sound objects dynamically for each finger."""
        sounds = {}
        for finger, freq in self.finger_frequencies.items():
            wave = self.generate_sine_wave(freq, duration=0.5)  # 0.5 second duration
            sounds[finger] = pygame.sndarray.make_sound(wave)
        return sounds

    @staticmethod
    def generate_sine_wave(frequency, duration, sample_rate=44100):
        """Generate a sine wave."""
        t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
        wave = 0.5 * np.sin(2 * np.pi * frequency * t)  # 0.5 to reduce amplitude

        # Convert to stereo: create a 2D array with identical left and right channels
        stereo_wave = np.stack((wave, wave), axis=-1)

        return (stereo_wave * 32767).astype(np.int16)  # Convert to 16-bit PCM

    def play_sound(self, finger):
        """Play the sound corresponding to the given finger."""
        if finger in self.sounds:
            self.sounds[finger].play()

    def detect_finger(self, landmarks):
        """Detect which finger is being pointed."""
        if not landmarks:
            return None

        finger_tips = {
            "thumb": 4,
            "index": 8,
            "middle": 12,
            "ring": 16,
            "pinky": 20
        }

        # Check if fingers are near the bottom of the frame (as a trigger area)
        for finger, tip_id in finger_tips.items():
            if landmarks[tip_id].y > 0.8:  # Bottom 20% of the screen
                return finger
        return None

    def process_frame(self, frame):
        """Process each video frame for hand detection and sound playing."""
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.hands.process(rgb_frame)

        height, width, _ = frame.shape

        # Draw trigger zone
        cv2.rectangle(frame, (0, int(height * 0.8)), (width, height), (200, 200, 200), -1)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                self.mp_draw.draw_landmarks(frame, hand_landmarks, self.mp_hands.HAND_CONNECTIONS)

                # Get finger tip landmarks
                landmarks = hand_landmarks.landmark
                finger = self.detect_finger(landmarks)
                if finger:
                    self.play_sound(finger)

        return frame

    def run(self):
        """Start the webcam and play sounds based on hand gestures."""
        cap = cv2.VideoCapture(0)

        try:
            while True:
                ret, frame = cap.read()
                if not ret:
                    break

                frame = cv2.flip(frame, 1)
                frame = self.process_frame(frame)

                # Add instructions
                cv2.putText(frame, "Move fingers to the bottom area to play sounds", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)

                # Display the frame
                cv2.imshow('Hand Sounds', frame)

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

        finally:
            cap.release()
            cv2.destroyAllWindows()
            pygame.mixer.quit()


# Run the HandSounds application
if __name__ == "__main__":
    hand_sounds = HandSounds()
    hand_sounds.run()


In [14]:
import cv2
import numpy as np
import pygame
import mediapipe as mp

class HandDrums:
    def __init__(self):
        # Initialize Mediapipe Hand detection
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(
            static_image_mode=False,
            max_num_hands=1,
            min_detection_confidence=0.7
        )
        self.mp_draw = mp.solutions.drawing_utils

        # Initialize Pygame mixer for sound playback
        pygame.mixer.init()

        # Drum characteristics for each finger
        self.finger_drums = {
            "thumb": {"duration": 0.2, "pitch": 100},   # Bass drum
            "index": {"duration": 0.1, "pitch": 300},   # Snare
            "middle": {"duration": 0.15, "pitch": 200}, # Tom
            "ring": {"duration": 0.12, "pitch": 400},   # Hi-hat (closed)
            "pinky": {"duration": 0.1, "pitch": 600},   # Cymbal
        }

        # Create sound objects for each finger
        self.sounds = self.generate_drums()

    def generate_drums(self):
        """Generate drum-like sounds dynamically for each finger."""
        sounds = {}
        for finger, params in self.finger_drums.items():
            wave = self.generate_drum_wave(duration=params["duration"], pitch=params["pitch"])
            sounds[finger] = pygame.sndarray.make_sound(wave)
        return sounds

    @staticmethod
    def generate_drum_wave(duration, pitch, sample_rate=44100):
        """Generate a drum-like sound using noise and exponential decay."""
        t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
        
        # Generate noise
        noise = np.random.uniform(-1, 1, len(t))
        
        # Apply exponential decay envelope
        envelope = np.exp(-5 * t)
        drum_wave = noise * envelope

        # Add a pitch element for tonal drum sounds
        tonal_wave = 0.5 * np.sin(2 * np.pi * pitch * t)
        drum_wave += tonal_wave * envelope

        # Convert to stereo: create a 2D array with identical left and right channels
        stereo_wave = np.stack((drum_wave, drum_wave), axis=-1)

        return (stereo_wave * 32767).astype(np.int16)  # Convert to 16-bit PCM

    def play_sound(self, finger):
        """Play the drum sound corresponding to the given finger."""
        if finger in self.sounds:
            self.sounds[finger].play()

    def detect_finger(self, landmarks):
        """Detect which finger is being pointed."""
        if not landmarks:
            return None

        finger_tips = {
            "thumb": 4,
            "index": 8,
            "middle": 12,
            "ring": 16,
            "pinky": 20
        }

        # Check if fingers are near the bottom of the frame (as a trigger area)
        for finger, tip_id in finger_tips.items():
            if landmarks[tip_id].y > 0.8:  # Bottom 20% of the screen
                return finger
        return None

    def process_frame(self, frame):
        """Process each video frame for hand detection and sound playing."""
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.hands.process(rgb_frame)

        height, width, _ = frame.shape

        # Draw trigger zone
        cv2.rectangle(frame, (0, int(height * 0.8)), (width, height), (200, 200, 200), -1)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                self.mp_draw.draw_landmarks(frame, hand_landmarks, self.mp_hands.HAND_CONNECTIONS)

                # Get finger tip landmarks
                landmarks = hand_landmarks.landmark
                finger = self.detect_finger(landmarks)
                if finger:
                    self.play_sound(finger)

        return frame

    def run(self):
        """Start the webcam and play drum sounds based on hand gestures."""
        cap = cv2.VideoCapture(0)

        try:
            while True:
                ret, frame = cap.read()
                if not ret:
                    break

                frame = cv2.flip(frame, 1)
                frame = self.process_frame(frame)

                # Add instructions
                cv2.putText(frame, "Move fingers to the bottom area to play drum sounds", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)

                # Display the frame
                cv2.imshow('Hand Drums', frame)

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

        finally:
            cap.release()
            cv2.destroyAllWindows()
            pygame.mixer.quit()


# Run the HandDrums application
if __name__ == "__main__":
    hand_drums = HandDrums()
    hand_drums.run()
